## 1. 安装依赖

In [ ]:
%pip install -q redis pandas openpyxl openai python-dotenv numpy

## 2. 配置环境变量

请在 `.env` 文件中配置以下变量，或直接在下方修改：

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv("../.env", override=True)

# ============================================================
# AZURE MANAGED REDIS 配置
# ============================================================
REDIS_HOST = "xle-redis2.eastus.redis.azure.net"
REDIS_PORT = 10000
REDIS_PASSWORD = os.getenv("AZURE_REDIS_PASSWORD", "<your-access-key-here>")  # 从 Azure Portal Access Keys 获取

# ============================================================
# AZURE OPENAI 配置 (用于生成 Embeddings)
# ============================================================
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
EMBEDDING_MODEL = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL", "text-embedding-ada-002")
EMBEDDING_DIMENSIONS = 1536  # ada-002 的维度

# ============================================================
# REDIS 索引配置
# ============================================================
INDEX_NAME = "product_index"

# 显示配置
print("📍 配置信息:")
print(f"   Redis Host: {REDIS_HOST}:{REDIS_PORT}")
print(f"   Redis Password: {'已设置' if REDIS_PASSWORD and REDIS_PASSWORD != '<your-access-key-here>' else '❌ 未设置'}")
print(f"   OpenAI Endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"   Embedding Model: {EMBEDDING_MODEL}")

## 3. 读取 Excel 数据

In [ ]:
import pandas as pd

# 读取 Excel 文件
excel_path = "./m1Id_tw_100.xlsx"
df = pd.read_excel(excel_path)

# 将列名转为小写
df.columns = [col.lower().strip() for col in df.columns]

print(f"✅ 读取了 {len(df)} 条记录")
print(f"📋 列名: {list(df.columns)}")
df.head()

## 4. 准备文档数据

In [ ]:
import uuid

def prepare_documents(df):
    """将 DataFrame 转换为文档列表"""
    documents = []
    
    for idx, row in df.iterrows():
        # 提取字段，处理空值
        url = str(row.get("url", "")) if pd.notna(row.get("url")) else ""
        name = str(row.get("name", "")) if pd.notna(row.get("name")) else ""
        m1id = str(row.get("m1id", "")) if pd.notna(row.get("m1id")) else ""
        spec = str(row.get("spec", "")) if pd.notna(row.get("spec")) else ""
        onlinedt = str(row.get("onlinedt", "")) if pd.notna(row.get("onlinedt")) else ""
        m1name = str(row.get("m1name", "")) if pd.notna(row.get("m1name")) else ""
        globalname = str(row.get("globalname", "")) if pd.notna(row.get("globalname")) else ""
        productline = str(row.get("productline", "")) if pd.notna(row.get("productline")) else ""
        aiproductline = str(row.get("aiproductline", "")) if pd.notna(row.get("aiproductline")) else ""
        
        # 构建用于搜索的内容文本
        content = f"产品名称: {name}\n规格: {spec}\n产品线: {productline}\nAI产品线: {aiproductline}"
        
        # 构建完整元数据
        metadata = f"m1id: {m1id}, m1name: {m1name}, globalname: {globalname}, url: {url}"
        
        document = {
            "id": str(uuid.uuid4()),
            "name": name,
            "m1id": m1id,
            "content": content,
            "metadata": metadata,
            "url": url,
            "productline": productline,
        }
        documents.append(document)
    
    return documents

documents = prepare_documents(df)
print(f"✅ 准备了 {len(documents)} 个文档")
print("\n📄 示例文档:")
documents[0]

## 5. 生成 Embeddings

In [ ]:
from openai import AzureOpenAI

# 初始化 Azure OpenAI 客户端
openai_client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2024-02-01",
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

def generate_embeddings(texts: list, batch_size: int = 100) -> list:
    """批量生成文本的 embedding 向量"""
    embeddings = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        response = openai_client.embeddings.create(
            input=batch,
            model=EMBEDDING_MODEL
        )
        batch_embeddings = [e.embedding for e in response.data]
        embeddings.extend(batch_embeddings)
        print(f"   生成 embeddings: {i + len(batch)}/{len(texts)}")
    
    return embeddings

# 提取内容文本
content_texts = [doc["content"] for doc in documents]

print("🔄 正在生成 embeddings...")
embeddings = generate_embeddings(content_texts)
print(f"\n✅ 生成了 {len(embeddings)} 个 embeddings，维度: {len(embeddings[0])}")

## 6. 连接 Azure Managed Redis

In [ ]:
import redis

# 连接到 Azure Managed Redis (TLS)
redis_client = redis.Redis(
    host=REDIS_HOST,
    port=REDIS_PORT,
    password=REDIS_PASSWORD,
    ssl=True,
    ssl_cert_reqs=None,  # 对于 Azure Managed Redis
    decode_responses=False,  # 向量数据需要 bytes
)

# 测试连接
try:
    pong = redis_client.ping()
    print(f"✅ Redis 连接成功! PING: {pong}")
    info = redis_client.info("server")
    print(f"   Redis 版本: {info.get('redis_version', 'unknown')}")
except Exception as e:
    print(f"❌ Redis 连接失败: {e}")

## 7. 创建 RediSearch 向量索引

In [ ]:
from redis.commands.search.field import TextField, TagField, VectorField

# 不同版本的 redis-py 路径不同
try:
    from redis.commands.search.indexDefinition import IndexDefinition, IndexType
except ImportError:
    from redis.commands.search.index_definition import IndexDefinition, IndexType

def create_vector_index(client, index_name: str, vector_dim: int = 1536):
    """创建 RediSearch 向量索引"""
    
    # 检查索引是否已存在
    try:
        client.ft(index_name).info()
        print(f"⚠️ 索引 '{index_name}' 已存在，删除并重建...")
        client.ft(index_name).dropindex(delete_documents=True)
    except:
        pass  # 索引不存在
    
    # 定义索引 schema
    schema = [
        TextField("name", weight=2.0),           # 产品名称，权重更高
        TextField("content"),                    # 内容文本
        TextField("metadata"),                   # 元数据
        TagField("productline"),                 # 产品线 (用于过滤)
        TextField("url"),                        # URL
        VectorField(
            "embedding",
            "HNSW",                              # 使用 HNSW 算法
            {
                "TYPE": "FLOAT32",
                "DIM": vector_dim,
                "DISTANCE_METRIC": "COSINE",     # 余弦相似度
                "M": 16,                          # HNSW 参数
                "EF_CONSTRUCTION": 200,
            }
        ),
    ]
    
    # 索引定义 - 使用 hash tag {doc} 确保所有 key 在同一个 slot
    # 这对于 Redis Cluster 模式(Enterprise)是必需的
    definition = IndexDefinition(
        prefix=["{doc}:"],
        index_type=IndexType.HASH
    )
    
    # 创建索引
    client.ft(index_name).create_index(
        fields=schema,
        definition=definition
    )
    
    print(f"✅ 索引 '{index_name}' 创建成功!")

# 创建索引
create_vector_index(redis_client, INDEX_NAME, EMBEDDING_DIMENSIONS)

## 8. 写入文档和向量

In [ ]:
import numpy as np

def upload_documents(client, documents: list, embeddings: list):
    """上传文档和向量到 Redis (兼容 Redis Cluster 模式)"""
    
    # 对于 Redis Cluster，使用 hash tag {doc} 确保所有 key 在同一个 slot
    # 这样 pipeline 才能正常工作
    
    pipeline = client.pipeline(transaction=False)  # 禁用事务模式，避免跨 slot 问题
    
    for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
        # 使用 hash tag {doc} - 所有 key 都会 hash 到同一个 slot
        key = f"{{doc}}:{doc['id']}"
        
        # 将 embedding 转换为 bytes
        embedding_bytes = np.array(embedding, dtype=np.float32).tobytes()
        
        # 存储为 Hash
        pipeline.hset(
            key,
            mapping={
                "id": doc["id"],
                "name": doc["name"],
                "content": doc["content"],
                "metadata": doc["metadata"],
                "productline": doc["productline"],
                "url": doc["url"],
                "embedding": embedding_bytes,
            }
        )
        
        # 每 50 条执行一次 (较小批次更稳定)
        if (i + 1) % 50 == 0:
            pipeline.execute()
            pipeline = client.pipeline(transaction=False)
            print(f"   已上传: {i + 1}/{len(documents)}")
    
    # 执行剩余的
    pipeline.execute()
    print(f"✅ 上传完成: {len(documents)} 个文档")

print("🔄 正在上传文档到 Redis...")
upload_documents(redis_client, documents, embeddings)

## 9. 验证索引

In [ ]:
# 获取索引信息
index_info = redis_client.ft(INDEX_NAME).info()

print("📊 索引信息:")
print(f"   索引名称: {index_info.get('index_name', INDEX_NAME)}")
print(f"   文档数量: {index_info.get('num_docs', 0)}")

# 索引大小可能是字符串，需要转换
inverted_sz = index_info.get('inverted_sz_mb', 0)
if isinstance(inverted_sz, str):
    inverted_sz = float(inverted_sz) if inverted_sz else 0
print(f"   索引大小: {inverted_sz:.2f} MB")

## 10. 向量相似度搜索

In [ ]:
from redis.commands.search.query import Query

def vector_search(client, query_text: str, index_name: str, top_k: int = 5):
    """
    执行向量相似度搜索
    
    Args:
        client: Redis 客户端
        query_text: 查询文本
        index_name: 索引名称
        top_k: 返回结果数量
    
    Returns:
        搜索结果列表
    """
    # 生成查询文本的 embedding
    response = openai_client.embeddings.create(
        input=[query_text],
        model=EMBEDDING_MODEL
    )
    query_embedding = response.data[0].embedding
    query_vector = np.array(query_embedding, dtype=np.float32).tobytes()
    
    # 构建 KNN 向量搜索查询
    # 语法: *=>[KNN K @field $param AS score]
    query = (
        Query(f"*=>[KNN {top_k} @embedding $query_vec AS score]")
        .sort_by("score")
        .return_fields("name", "content", "metadata", "productline", "url", "score")
        .dialect(2)
    )
    
    # 执行搜索
    results = client.ft(index_name).search(
        query,
        query_params={"query_vec": query_vector}
    )
    
    return results

# 测试搜索
test_query = "工业电脑"
print(f"🔍 搜索: '{test_query}'\n")

results = vector_search(redis_client, test_query, INDEX_NAME, top_k=5)

print(f"找到 {results.total} 个结果:\n")
for i, doc in enumerate(results.docs, 1):
    score = float(doc.score) if hasattr(doc, 'score') else 0
    name = doc.name.decode() if isinstance(doc.name, bytes) else doc.name
    content = doc.content.decode() if isinstance(doc.content, bytes) else doc.content
    
    print(f"--- 结果 {i} (相似度: {1-score:.4f}) ---")
    print(f"名称: {name}")
    print(f"内容: {content[:200]}...")
    print()

## 11. 混合搜索 (向量 + 过滤)

In [ ]:
def hybrid_search(client, query_text: str, index_name: str, 
                  productline_filter: str = None, top_k: int = 5):
    """
    混合搜索: 向量搜索 + 元数据过滤
    """
    # 生成查询 embedding
    response = openai_client.embeddings.create(
        input=[query_text],
        model=EMBEDDING_MODEL
    )
    query_embedding = response.data[0].embedding
    query_vector = np.array(query_embedding, dtype=np.float32).tobytes()
    
    # 构建带过滤的查询
    if productline_filter:
        # 过滤特定产品线
        query_str = f"(@productline:{{{productline_filter}}})=>[KNN {top_k} @embedding $query_vec AS score]"
    else:
        query_str = f"*=>[KNN {top_k} @embedding $query_vec AS score]"
    
    query = (
        Query(query_str)
        .sort_by("score")
        .return_fields("name", "content", "productline", "score")
        .dialect(2)
    )
    
    results = client.ft(index_name).search(
        query,
        query_params={"query_vec": query_vector}
    )
    
    return results

# 测试混合搜索
test_query = "嵌入式系统"
print(f"🔍 混合搜索: '{test_query}'\n")

results = hybrid_search(redis_client, test_query, INDEX_NAME, top_k=3)

for i, doc in enumerate(results.docs, 1):
    score = float(doc.score) if hasattr(doc, 'score') else 0
    name = doc.name.decode() if isinstance(doc.name, bytes) else doc.name
    
    print(f"{i}. {name} (相似度: {1-score:.4f})")

## 12. RAG 问答函数

In [ ]:
def rag_query(question: str, top_k: int = 3) -> str:
    """
    RAG 问答: 检索相关文档并生成回答
    
    Args:
        question: 用户问题
        top_k: 检索文档数量
    
    Returns:
        AI 生成的回答
    """
    # 1. 向量搜索检索相关文档
    results = vector_search(redis_client, question, INDEX_NAME, top_k=top_k)
    
    # 2. 构建上下文
    context_parts = []
    sources = []
    
    for doc in results.docs:
        name = doc.name.decode() if isinstance(doc.name, bytes) else doc.name
        content = doc.content.decode() if isinstance(doc.content, bytes) else doc.content
        metadata = doc.metadata.decode() if isinstance(doc.metadata, bytes) else doc.metadata
        
        context_parts.append(f"产品: {name}\n{content}\n{metadata}")
        sources.append(name)
    
    context = "\n\n---\n\n".join(context_parts)
    
    # 3. 使用 LLM 生成回答
    system_prompt = """你是一个产品专家助手。根据提供的产品信息回答用户问题。
如果信息不足以回答问题，请诚实说明。请用中文回答。"""
    
    user_prompt = f"""根据以下产品信息回答问题：

产品信息：
{context}

问题：{question}

请提供详细的回答："""
    
    response = openai_client.chat.completions.create(
        model="gpt-4.1",  # 或您的部署名称
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.7,
        max_tokens=500
    )
    
    answer = response.choices[0].message.content
    
    return {
        "answer": answer,
        "sources": sources
    }

# 测试 RAG 问答
question = "请推荐一款适合工业自动化的产品"
print(f"❓ 问题: {question}\n")

result = rag_query(question)

print(f"💬 回答:\n{result['answer']}")
print(f"\n📚 参考来源: {', '.join(result['sources'])}")

## 13. 清理资源 (可选)

In [ ]:
# 如需删除索引和数据，取消下面的注释并运行

# redis_client.ft(INDEX_NAME).dropindex(delete_documents=True)
# print(f"✅ 索引 '{INDEX_NAME}' 已删除")

print("ℹ️ 如需清理，请取消上面代码的注释")

## 📋 总结

本 notebook 演示了：

1. ✅ 从 Excel 读取产品数据
2. ✅ 使用 Azure OpenAI 生成 embeddings
3. ✅ 连接 Azure Managed Redis (Enterprise + RediSearch)
4. ✅ 创建 HNSW 向量索引
5. ✅ 上传文档和向量
6. ✅ 执行向量相似度搜索
7. ✅ 实现 RAG 问答

### 下一步

- 将搜索功能集成到 Voice Assistant
- 添加更多过滤条件
- 优化 embedding 和检索策略